In [ ]:
# install some libraries if not installed already
!pip install --upgrade scikit-image 
!pip install imgaug
!pip install pandas==0.23.4
!pip install pydicom

In [ ]:
# get the dataset if you don't have it already or on Colab - 1
import os
!git clone https://github.com/myleott/mnist_png.git

In [ ]:
# get the dataset if you don't have it already or on Colab - 2
os.chdir('mnist_png')

In [ ]:
# get the dataset if you don't have it already or on Colab - 3
!tar -xzf mnist_png.tar.gz

In [ ]:
# Download and unzip pneumonia detection data if you're on Colab or don't have it already on Biowulf - 1
ROOT_DIR = '../pn_chest_xray'
if not os.path.exists(ROOT_DIR):
    os.makedirs(ROOT_DIR)
os.chdir(ROOT_DIR)

In [ ]:
# Download and unzip pneumonia detection data if you're on Colab or don't have it already on Biowulf - 2
!wget --no-check-certificate -r 'https://docs.google.com/uc?export=download&id=1ertt7A696f8HC0qk-yqmSoTZKCsp2XWU' -O stage_1_train_images.zip

In [ ]:
# Download and unzip pneumonia detection data if you're on Colab or don't have it already on Biowulf - 3
!wget --no-check-certificate -r 'https://docs.google.com/uc?export=download&id=1ppgqzDVa9hQYbxgjV_k7AUUOKTw7ca9Q' -O stage_1_test_images.zip

In [ ]:
# Download and unzip pneumonia detection data if you're on Colab or don't have it already on Biowulf - 4
!wget --no-check-certificate -r 'https://docs.google.com/uc?export=download&id=1yI8jJw9DGMeZeMgHaoZBUeuebt11qe0v' -O stage_1_train_labels.csv.zip

In [ ]:
# Download and unzip pneumonia detection data if you're on Colab or don't have it already on Biowulf - 5
!wget --no-check-certificate -r 'https://docs.google.com/uc?export=download&id=1lpa5bsCKOgesx1zP7h5UIDwN4FzvjooD' -O stage_1_detailed_class_info.csv.zip

In [ ]:
# Download and unzip pneumonia detection data if you're on Colab or don't have it already on Biowulf - 6
# unzipping takes a few minutes
!unzip -q -o stage_1_test_images.zip -d stage_1_test_images
!unzip -q -o stage_1_train_images.zip -d stage_1_train_images
!unzip -q -o stage_1_train_labels.csv.zip
!unzip -q -o stage_1_detailed_class_info.csv.zip

In [ ]:
# Download and unzip pneumonia detection data if you're on Colab or don't have it already on Biowulf - 7
# fix some permission issues
!chmod 644 stage_1_detailed_class_info.csv
!chmod 644 stage_1_train_labels.csv

In [ ]:
# import some libraries we will use
import os, sys, glob, csv
import pydicom

import matplotlib.pyplot as plt
import numpy as np

from skimage import io

In [ ]:
# MNIST handwritten digits

In [ ]:
mnist_base_dir = '../mnist_png/mnist_png' # or point to appropriate directory if you're on Biowulf and already have the data

In [ ]:
# directories for training and testing sets 
mnist_base_dir_tr = os.path.join(mnist_base_dir, 'training/')
mnist_base_dir_te = os.path.join(mnist_base_dir, 'testing/')

In [ ]:
# list '1' images
mnist_ones_tr = glob.glob(os.path.join(mnist_base_dir_tr, '1', '*.png'))
mnist_ones_te = glob.glob(os.path.join(mnist_base_dir_te, '1', '*.png'))

In [ ]:
# display 10 examples
for idx, imf in enumerate(mnist_ones_tr):
    plt.figure()
    plt.imshow(io.imread(imf), cmap='gray')
    
    if idx >=10:
        break

In [ ]:
# Task 1: try visualize test cases and other digits categories

In [ ]:
# Task 2: try to randomize which 10 images you visualize

In [ ]:
# Chest x-ray pneumonia detection dataset

In [ ]:
# If you're on Colab - 1
!git clone https://github.com/khcs/NIH-FAES-BIOF-399-Spring-2019.git

In [ ]:
# If you're on Colab - 2
sys.path.append("NIH-FAES-BIOF-399-Spring-2019/week-02")

In [ ]:
# chest x-ray root directory (that you downloaded from week-01)
chestpnx_dir = '../pn_chest_xray' # or point to the right directory
stage_num = '1'
test_only_part = True

In [ ]:
# import custom-written functions
import image_bbox_loader
import box_utils

In [ ]:
# load images, bounding-boxes, labels using the custom-written functions
image_bbox_label_list = image_bbox_loader.load_image_bbox_label_list(chestpnx_dir,
                                                                     stage_num, test_only_part)

In [ ]:
# show one example image
for imf, boxes, labels in image_bbox_label_list:
    img = pydicom.dcmread(imf).pixel_array
    plt.imshow(img, cmap='gray')
    break

In [ ]:
# import imgaug package for visuzlizing bounding-boxes
from imgaug import augmenters as iaa
import imgaug as ia

In [ ]:
# show one example image with bounding boxes
for imf, boxes, labels in image_bbox_label_list:
    img = pydicom.dcmread(imf).pixel_array
    img2 =  np.stack((img,)*3, axis=-1)
    
    for boxi_ in boxes: # batch_size=1
        
        boxi = box_utils.center_form_to_corner_form(
               box_utils.min_wh_form_to_center_form(boxi_)).numpy()
        
        bbi = ia.BoundingBox(x1=boxi[0],
                             x2=boxi[2],
                             y1=boxi[1],
                             y2=boxi[3])
        img2 = bbi.draw_on_image(img2, thickness=6)
    
    plt.imshow(img2)
    break

In [ ]:
# Task 3: try to visualize 10 random examples of chest x-ray with pneumonia